In [1]:
%pip install pyspark

  Using cached pyspark-3.5.3.tar.gz (317.3 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840672 sha256=80aa6ab6d79b5bb028f0abe6b5542391e02981ceb781810d57a24ba706616163
  Stored in directory: c:\users\lenovo\appdata\local\pip\cache\wheels\57\01\5d\dd344b64ae939cd8ea67ffa0abd6aed72f14b1165228e281a0
Successfully built pyspark
Note: you may need to restart the kernel to use updated packages.


In [18]:
from pyspark.sql import SparkSession

# Initialize Spark session with increased memory
spark = SparkSession.builder \
    .appName("WARK Data Pipeline - Predict Subject Area") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()


# Load CSV data into a Spark DataFrame
data_path = "C:/Users/Public/Documents/My/DataSci/DSDE_Final_Project_WARK/Data_Aj/2/joined_2018-2023.csv"
df = spark.read.csv(data_path, header=True, inferSchema=True)

# Show the data schema
df.printSchema()
df.show(5)

root
 |-- citation_title: string (nullable = true)
 |-- abstracts: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- affiliations: string (nullable = true)
 |-- classifications: string (nullable = true)
 |-- subject_area_name: string (nullable = true)
 |-- subject_area_code: string (nullable = true)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      citation_title|           abstracts|             authors|        affiliations|     classifications|   subject_area_name|   subject_area_code|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Public health and...|                   -|Pongpirul Krit; L...|Stanford Universi...|ASJC: 2700; SUBJA...|      Medicine (all)|                2700|
|Flexible Printed ...|© 2018 The Instit...|Pratumsiri Teerap...|Chulalongkorn Uni..

In [19]:
# Drop the redundant column
df = df.drop('subject_area_code')

# Show the data schema
df.printSchema()
df.show(5)


root
 |-- citation_title: string (nullable = true)
 |-- abstracts: string (nullable = true)
 |-- authors: string (nullable = true)
 |-- affiliations: string (nullable = true)
 |-- classifications: string (nullable = true)
 |-- subject_area_name: string (nullable = true)

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|      citation_title|           abstracts|             authors|        affiliations|     classifications|   subject_area_name|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Public health and...|                   -|Pongpirul Krit; L...|Stanford Universi...|ASJC: 2700; SUBJA...|      Medicine (all)|
|Flexible Printed ...|© 2018 The Instit...|Pratumsiri Teerap...|Chulalongkorn Uni...|ASJC: 2208\2504; ...|Electrical and El...|
|Parametric study ...|© 2018 Elsevier L...|Phuakpunk Kiattik...|Chulalongkorn Uni...|CPX

In [20]:
from pyspark.sql.functions import col, concat_ws

# Replace null or missing values
df_cleaned = df.fillna({
    'citation_title': '-',
    'abstracts': '-',
    'authors': '-',
    'affiliations': '-',
    'classifications': '-',
    'subject_area_name': '-',
})

# Combine relevant fields into a single feature
df_cleaned = df_cleaned.withColumn(
    "features_combined", 
    concat_ws("||", col("citation_title"), col("abstracts"), col("authors"), col("affiliations"), col("classifications"))
)

# Show the combined features
df_cleaned.select("features_combined", "subject_area_name").show(5, truncate=False)


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [21]:
from pyspark.ml.feature import Tokenizer, CountVectorizer, StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col

# Ensure no null values in required columns
df_cleaned = df_cleaned.fillna({"features_combined": "-", "subject_area_name": "-"})

# Tokenize the combined features
tokenizer = Tokenizer(inputCol="features_combined", outputCol="tokens")

# Convert tokens into numerical features
vectorizer = CountVectorizer(inputCol="tokens", outputCol="features", vocabSize=5000)

# Encode target labels into numerical format
label_indexer = StringIndexer(inputCol="subject_area_name", outputCol="label")

# Create a preprocessing pipeline
preprocessing_pipeline = Pipeline(stages=[tokenizer, vectorizer, label_indexer])

# Fit and transform the data
preprocessed_data = preprocessing_pipeline.fit(df_cleaned).transform(df_cleaned)

# Verify the transformed data
preprocessed_data.select("features", "label").show(5)



+--------------------+------+
|            features| label|
+--------------------+------+
|(5000,[1,2,6,10,1...|   1.0|
|(5000,[0,1,2,3,4,...|2615.0|
|(5000,[0,1,2,3,4,...| 119.0|
|(5000,[0,1,2,3,4,...| 191.0|
|(5000,[0,1,2,3,4,...| 230.0|
+--------------------+------+
only showing top 5 rows



In [23]:
# Check if features and label columns are present
preprocessed_data.select("features", "label").show(5)

# Split the dataset into training and testing sets
train_data, test_data = preprocessed_data.randomSplit([0.7, 0.3], seed=42)

# Sample a fraction of the data
train_data = train_data.sample(fraction=0.1, seed=42)
test_data = test_data.sample(fraction=0.1, seed=42)


print(f"Training Data Count: {train_data.count()}, Test Data Count: {test_data.count()}")


+--------------------+------+
|            features| label|
+--------------------+------+
|(5000,[1,2,6,10,1...|   1.0|
|(5000,[0,1,2,3,4,...|2615.0|
|(5000,[0,1,2,3,4,...| 119.0|
|(5000,[0,1,2,3,4,...| 191.0|
|(5000,[0,1,2,3,4,...| 230.0|
+--------------------+------+
only showing top 5 rows

Training Data Count: 1378, Test Data Count: 610


In [1]:
from pyspark.ml.classification import LogisticRegression

# Initialize the logistic regression model
lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=10)

# Train the model
lr_model = lr.fit(train_data)

# Evaluate the model on the test data
predictions = lr_model.transform(test_data)

# Display predictions
predictions.select("features", "label", "prediction").show(5)


AssertionError: 

In [ ]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Evaluate accuracy
evaluator = MulticlassClassificationEvaluator(
    labelCol="label", predictionCol="prediction", metricName="accuracy"
)

accuracy = evaluator.evaluate(predictions)
print(f"Accuracy: {accuracy:.2f}")


In [ ]:
# Save the trained model
lr_model.save("logistic_regression_model")

# Save the preprocessing pipeline
preprocessing_pipeline.save("preprocessing_pipeline")


In [ ]:
from pyspark.ml.pipeline import PipelineModel
from pyspark.ml.classification import LogisticRegressionModel

# Load the pipeline and model
loaded_pipeline = PipelineModel.load("preprocessing_pipeline")
loaded_model = LogisticRegressionModel.load("logistic_regression_model")

# Process new data
new_data = spark.createDataFrame([{
    "citation_title": "New Research in AI",
    "abstracts": "Deep learning advances...",
    "authors": "Author A; Author B",
    "affiliations": "University of XYZ",
    "classifications": "ASJC: 1700; SUBJABBR: COMP",
    "subject_area_name": "Unknown"
}])

new_data_cleaned = loaded_pipeline.transform(new_data)
new_predictions = loaded_model.transform(new_data_cleaned)
new_predictions.select("features", "prediction").show()
